## Data analysis
https://www.r-graph-gallery.com/101_Manhattan_plot.html

In [1]:
install.packages("qqman")
library(qqman)

setwd("target_dir/")
 
file_list <- list.files()
 
for (file in file_list){
       
  # if the merged dataset doesn't exist, create it
  if (!exists("dataset")){
    dataset <- read.table(file, header=TRUE, sep="\t")
  }
   
  # if the merged dataset does exist, append to it
  if (exists("dataset")){
    temp_dataset <-read.table(file, header=TRUE, sep="\t")
    dataset<-rbind(dataset, temp_dataset)
    rm(temp_dataset)
  }
 
}

#another way to merge files

dataset <- do.call("rbind",lapply(file_list
FUN=function(files){read.table(files,
header=TRUE, sep="\t")}))


manhattan(gwasResults, chr="CHR", bp="BP", snp="SNP", p="P" )

# Let's highlight them, with a bit of customization on the plot
manhattan(gwasResults, highlight = snpsOfInterest)
#Annotate the SNPs of interest
manhattan(gwasResults, annotatePval = 0.01)
#Make a qqplot
qq(gwasResults$P)

ERROR: Error in parse(text = x, srcfile = src): <text>:27:1: unexpected symbol
26: dataset <- do.call("rbind",lapply(file_list
27: FUN
    ^
